In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import re
import requests
import bs4
import numpy as np
import time

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [12]:
YEAR_START = 2021 
YEAR_END = 2021

filepath = "race_result_2021season.csv"

In [13]:
# Set up selenium to use Firefox
options = Options()
options.headless = True #No need to open a browser window
# driver = webdriver.Firefox(options=options)

# Example of manaully specifying the WebDriver's location: 
driver = webdriver.Firefox(executable_path="../Others/geckodriver.exe",options=options) #Windows
# driver = webdriver.Firefox(executable_path="../Others/geckodriver",options=options) #Linux

# Fetch the page
#driver.get('http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20100101/ST')

In [14]:
from bs4 import NavigableString

def get_sibling(tag,previous=False):
    if previous:
        sibling = tag.previous_sibling
        while isinstance(sibling, NavigableString):
            sibling = sibling.previous_sibling
    else:
        sibling = tag.next_sibling
        while isinstance(sibling, NavigableString):
            sibling = sibling.next_sibling        
    return sibling

In [15]:
def table(url, date, match):
    driver.get(url)
    
    # Is there anything?
    if driver.page_source.find("No information.") != -1:
        return []
    
    # Wait 20 secs so that the dynamic content has time to load.
    # Proceed to next date if page doesn't load.
    try:
        wait = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "f_fs13")))
    except:
        return []
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table', class_ = 'f_tac table_bd draggable')
    cdr_table = soup.find('tbody', class_ = 'f_fs13')
    # output_list = []
    data = []
    horseid_list = []
    
    #horseid
    horses = soup.find_all(href=re.compile("HorseId"))
    
    #prize money
    prize_money = soup.find_all("td")
    pm = 0
    for m in prize_money:
        prize_money = m.text.strip()
        if ("HK$ " in prize_money):
            pm = prize_money[4:]
            
    if isinstance(table, bs4.element.Tag):
        rows = table.find_all('tr')
        cdr_rows = cdr_table.find_all('tr')
        cdr_data = []
        
        
        for cdr_row in cdr_rows[1:2]:
            cdr_cols = cdr_row.find_all('td')
            cdr_cols = [ele.text.strip().replace('\n','') 
                          .replace(' '*20,' ') for ele in cdr_cols]
            cdr_data.append([ele for ele in cdr_cols if ele])
            cdr = cdr_data[0]
            cdr = cdr[0]
        
        #horseid
        for horse in horses:
            output = [horse.text.strip()]
            horseId = horse['href']
            if ("HorseId" in horseId):
                horseId = horseId[53:]
                output.append(horseId)
                horseid_list.append(output)

        for row in rows[1:]:
            cols = row.find_all('td')
            cols = [ele.text.strip().replace('\n','') 
                          .replace(' '*20,' ')
                          .replace("-", " ") for ele in cols]
            cols.append(str(date))
            cols.append(str(match))
            cols.append(cdr)
            cols.append(pm)
            data.append([ele for ele in cols])
        data = np.concatenate([data, horseid_list], axis=1)
    # output_list.append(data)

    return data

In [16]:
def scrape_horses(url, date):
    # Function to access a page and save all horses into a list

    # Fetch the page
    driver.get(url)
    
    # Is there anything?
    if driver.page_source.find("No information.") != -1:
        return []
    
    # Wait 20 secs so that the dynamic content has time to load.
    # Proceed to next date if page doesn't load.
    try:
        wait = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "f_fs13")))
    except:
        return []
    
    # Load the page into BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find all tags with href containing "HorseId"
    horses = soup.find_all(href=re.compile("HorseId"))

    # 'output_list' is the whole table
    # 'output' is a single row
    output_list = []
    
    # Loop through horses
    for horse in horses:
        # Get the horse name
        output = [horse.text.strip()]
        # This while loop fetch all remaining fields in a row
        a = get_sibling(horse.parent)
        
        while a != None:
            output.append(a.text
                          .strip()
                          # The last two lines are for running positions
                          .replace('\n','') 
                          .replace(' '*20,' ')
                          .replace("-", " ")
                         )
            a = get_sibling(a)
        
        # Append each row to the output list
        output_list.append(output)

    return output_list

In [18]:
url_front = "http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/"

import csv
with open(filepath, 'w', newline='') as csvfile:
    mywriter = csv.writer(csvfile)
    mywriter.writerow(['Plc.', 'Horse No.', 'Horse', 'Jockey', 'Trainer', 'Actual Wt.',
  'Declar. Horse Wt.', 'Draw', 'LBW', 'RunningPosition', 'Finish Time', 'Win Odds', 'date', 'match', 'cdr', 'prize_money', 'horse_name', 'horseid'])
    #Copy the loop from above and incorporate the csv-saving code
    for year in range(YEAR_START, YEAR_END+1):
        for month in range(1, 13):
            start = time.time()
            for day in range(1, 32):
                
                #Convert month and day to 2-digit representation
                month_2d = '{:02d}'.format(month)
                day_2d = '{:02d}'.format(day)
                
                #Full URL of data source
                date = str(year) + month_2d + day_2d
                url = url_front + date
                
                
                #Call scrape_horses function to fetch and check if is a race day
                content = scrape_horses(url, date)
                content = np.array(content)
                
                # Proceed only save if there is something in content
                if len(content) > 0:
                    
                    for match in range(1,13):
                        time.sleep(5)
                        url = url_front + date + '/' + str(match)
                        
                        content = table(url, date, match) # what really write into the csv
                        content = np.array(content)
                        if len(content) > 0:
                            mywriter.writerows(content)
                            
            print(year, month, (time.time() - start)/60)

2021 1 11.258579587936401
2021 2 9.879392131169636
2021 3 11.528138709068298
2021 4 1.563841700553894
2021 5 0.3354024569193522
2021 6 0.30183908144632976
2021 7 0.30592814286549885
2021 8 0.28739705483118694
2021 9 0.302562681833903
2021 10 0.2788531104723612
2021 11 0.2721358299255371
2021 12 0.30392105182011925
